# Generate Manhattan Plots and LM plots for 1 SNP

## IMPORTS & FUNCTIONS

In [3]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
sns.set_theme(style="whitegrid")
pheno = pd.read_csv("../Ressources/pheno.csv")
pheno = pheno[["HybID","NSFTVID","Seed.number.per.panicle","Amylose.content","Straighthead.suseptability"]]
geno = pd.read_csv("../Ressources/geno.csv")

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
def getModel(X,Y):
    X2 = sm.add_constant(X)
    mod = sm.OLS(Y,X2,missing="raise")
    fii = mod.fit()
    return mod,fii

def getPValues(fii):
    return fii.summary2().tables[1]["P>|t|"]

def plotManhattan(genes,pvalues,chromosomes,colname,title="test plot",ymax=15):
    """
    Generate and save manhattan plot
    
    genes: list of genes indexes
    pvalues: list of the p-values corresponding to the genes
    chromosomes: list of the chromosomes corresponding to the genes
    title: title of the plot
    savepath: path where the plot is saved
    """
    df = pd.DataFrame(
        {
            'gene' : genes,
            'pvalue' : pvalues,
            'chromosome' : chromosomes
        })
    
    # -log_10(pvalue)
    nbchrom = len(df.chromosome.unique())
    df['minuslog10pvalue'] = -np.log10(df.pvalue)
    df.chromosome = df.chromosome.astype('category')
    df.chromosome = df.chromosome.cat.set_categories(['ch-%i' % i for i in range(1,nbchrom+1)], ordered=True)
    df = df.sort_values('chromosome')

    # How to plot gene vs. -log10(pvalue) and colour it by chromosome?
    df['ind'] = range(len(df))
    df_grouped = df.groupby(('chromosome'))

    # manhattan plot
    fig = plt.figure(figsize=(14, 8)) # Set the figure size
    
    
    ax = fig.add_subplot(111)
    colors = ['gold','deepskyblue','hotpink', 'limegreen']
    x_labels = []
    x_labels_pos = []
    for num, (name, group) in enumerate(df_grouped):
        group.plot(kind='scatter', x='ind', y='minuslog10pvalue',color=colors[num % len(colors)], ax=ax)
        x_labels.append(name)
        x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
    ax.set_xticks(x_labels_pos)
    ax.set_xticklabels(x_labels)

    # set axis limits
    bonfcorrLimit = -np.log10(0.01/df.shape[0])
    print(bonfcorrLimit)
    ax.set_xlim([0, len(df)])
    #ax.set_ylim([0, max(df["minuslog10pvalue"].max(),bonfcorrLimit+0.5)])
    ax.set_ylim([0, ymax])

    # x axis label
    ax.set_xlabel('Chromosome')
    
    # Significant genes separation with bonferroni correction
    plt.axhline(y = bonfcorrLimit, color = 'r', linestyle = '-', label='separation with Bonferroni correction')

    # show the graph
    plt.title(title, y=-0.2)
    plt.legend()
    savepath=f"manhattan_{colname.replace('.','_')}.png"
    plt.savefig('manhattan_test.png')
    plt.show()

def plotPhenoGeno(i):
    Y = pd.Series(list(pheno["Seed.number.per.panicle"]),name="Y")
    X = pd.Series(list(genoMat.loc[i]),name="X")
    df=pd.concat([X,Y],axis=1)
    df.dropna(inplace=True)
    mod,fii = getModel(df["X"],df["Y"])
    pv = getPValues(fii)["X"]
    plt.scatter(X,Y)
    print(fii.params)
    intercept,slope = fii.params.get("const",0),fii.params.get("X")
    plt.plot(X, X*slope + intercept, 'r')
    print(slope,intercept,pv)
    print(-np.log10(pv))

def plotPhenoGeno2(i,colName):
    Y = pd.Series(list(pheno[colName]),name="Y")
    X = pd.Series(list(genoMat.loc[i]),name="X")
    df=pd.concat([X,Y],axis=1)
    df.dropna(inplace=True)

    fig, axs = plt.subplots(ncols=2, figsize=(8,6))
    # Draw a nested violinplot and split the violins for easier comparison
    colorDict = {
        0:"deepskyblue",
        1:"hotpink",
        2:"limegreen"
    }
    sns.violinplot(data=df, x="X", y="Y",
                   split=True, inner="quart", linewidth=2,ax=axs[0],palette=colorDict)
    g = sns.regplot(
    data=df, x="X", y="Y",
    scatter=True, truncate=False, order=1,scatter_kws={"color":"black"},ax=axs[1]
    )
    axs[0].set_ylabel(colName.replace("."," "))
    axs[1].set_ylabel("")
    axs[0].set_xlabel("SNP value")
    axs[1].set_xlabel("SNP value")
    fig.suptitle(f'SNP n°{i+1} analysis with linear model', fontsize=16)
    
    # Calculate number of obs per group & median to position labels
    medians = df.groupby(['X']).median().sort_index().values
    nobs = df['X'].value_counts().sort_index().values
    nobs = [str(x) for x in nobs.tolist()]
    nobs = ["n: " + i for i in nobs]
    
    # Add text to the figure
    pos = range(len(nobs))
    for tick, label in zip(pos, axs[0].get_xticklabels()):
       axs[0].text(pos[tick], medians[tick] + 0.03, nobs[tick],
                horizontalalignment='center',
                size='small',
                color='w',
                weight='semibold')
    plt.show()

def plotPhenoGeno3(i):
    Y = pd.Series(list(pheno["Seed.number.per.panicle"]),name="Y")
    X = pd.Series(list(genoMat.loc[i]),name="X")
    df=pd.concat([X,Y],axis=1)
    df.dropna(inplace=True)
    
    g = sns.regplot(
    data=df, x="X", y="Y",
    scatter=False, truncate=False, order=1, color=".2",
    )

    #g.set_axis_labels("Genotype SNP", "Seed Number Per Panicle")
    g.set(xlabel='common xlabel', ylabel='common ylabel')
    plt.show()

def computePValues(phenoColumnName,genoMat):
    pvalues = []
    Y = pd.Series(list(pheno[phenoColumnName]),name="Y")
    for i in tqdm(range(genoMat.shape[0])):
        X = pd.Series(list(genoMat.loc[i]),name="X")
        df=pd.concat([X,Y],axis=1)
        df.dropna(inplace=True)
        mod,fii = getModel(df["X"],df["Y"])
        pvalues.append(getPValues(fii)["X"])
    return pvalues

def saveMdf(genes,pvalues,chromosomes,name="manhattanDF"):
    df = pd.DataFrame(
        {
            'gene' : genes,
            'pvalue' : pvalues,
            'chromosome' : chromosomes
        })
    df.to_csv(f"{name}.csv")
    return df

def saveAllManhattanDf(pheno,geno):
    genes = geno["marker"]
    chromosomes = pd.Series([f"ch-{i}" for i in geno["chrom"]])
    genoMat = geno[[str(i) for i in range(1,414)]]
    res = {}
    for col in pheno.columns[2:]:
        pvalues = computePValues(col,genoMat)
        res[col] = saveMdf(genes,pvalues,chromosomes,name=f"manhattan_{col.replace('.','_')}")
    return res

def getBestPValues(pvalues,maxpvalue,minpvalue):
    m = []
    for i in range(len(pvalues)):
        mtmp = -np.log10(pvalues[i])
        if mtmp>minpvalue and mtmp<maxpvalue:
            m.append((mtmp,i))
    m = sorted(m, key = lambda x: x[0],reverse=True)
    return m

## Available DATA

In [ ]:
pheno

In [ ]:
geno

### Répartition des valeurs du dataset

In [ ]:
names = ["homozygotes dominants","hétérozygotes","homozygotes récessifs"]
all = genoMat.shape[0]*genoMat.shape[1]
for n in [0,1,2]:
    x = np.count_nonzero(genoMat.values==n)
    print(f"Dans genoMat il y a {x} allèles {names[n]} ({round(x*100/all,2)}%)")

x = np.count_nonzero(genoMat.isnull().values==1)
print(f"Dans genoMat il y a {x} allèles non définis ({round(x*100/all,2)}%)")

## Generate and save p-values for each characteristic

In [ ]:
#saveAllManhattanDf(pheno,geno)

# Generate and save Manhattan plots

In [ ]:
#ymax = [14,70,35] #max values for the plots
cols = pheno.columns[2:]

## Seed number per panicle

- ne pas oublier de mentionner qu'il y a des SNP qui n'ont que des 0 ou des 2
- ne pas oublier de mentionner qu'il n'y a pas beaucoup d'allèles hétérozygotes dans genoMat (0.04%)

In [ ]:
df1 = pd.read_csv(f"manhattan_{cols[0].replace('.','_')}.csv")
df1.head()

In [ ]:
plotManhattan(
        df1['gene'],
        df1['pvalue'],
        df1['chromosome'],
        title=f"Manhattan plot for {cols[0]}",
        colname=cols[0],
        ymax=15)

In [ ]:
bestpvalues = getBestPValues(df1["pvalue"],15,10)[:5]
for pv, idx in bestpvalues:
    plotPhenoGeno2(idx,cols[0])

### Les 5 meilleures SNP pour le seed number

In [ ]:
print([i[1] for i in bestpvalues])

## Amylose content

In [ ]:
df2 = pd.read_csv(f"manhattan_{cols[1].replace('.','_')}.csv")
df2.head()

In [1]:
plotManhattan(
        df2['gene'],
        df2['pvalue'],
        df2['chromosome'],
        title=f"Manhattan plot for {cols[1]}",
        colname=cols[1],
        ymax=60)

NameError: name 'plotManhattan' is not defined

In [ ]:
bestpvalues = getBestPValues(df2["pvalue"],60,40)[:5]
for pv, idx in bestpvalues:
    plotPhenoGeno2(idx,cols[1])

### Les 5 meilleures SNP pour amylose content

### Les 5 meilleures SNP pour le seed number

In [ ]:
print([i[1] for i in bestpvalues])

## Straighthead susceptability

In [ ]:
df3 = pd.read_csv(f"manhattan_{cols[2].replace('.','_')}.csv")
df3.head()

In [ ]:
plotManhattan(
        df3['gene'],
        df3['pvalue'],
        df3['chromosome'],
        title=f"Manhattan plot for {cols[2]}",
        colname=cols[2],
        ymax=40)

In [ ]:
bestpvalues = getBestPValues(df3["pvalue"],35,25)[:5]
for pv, idx in bestpvalues:
    plotPhenoGeno2(idx,cols[2])

### Les 5 meilleures SNP pour le seed number

In [ ]:
print([i[1] for i in bestpvalues])